In [2]:
import pandas as pd

In [23]:
url = "https://drive.google.com/file/d/1U34r_SnEZNpIPXM-_lRDjlin3vbTN_sL/view?usp=drive_link"

In [20]:
pd.set_option('display.max_columns', None)

In [25]:
base = pd.read_csv(url, delimiter = ";")
base2 = pd.read_csv("Varmod_SALAAN_2021.csv", delimiter = ";")

ParserError: Error tokenizing data. C error: Expected 23 fields in line 17, saw 25


In [21]:
base.head()

,A6,A17,A38,CPFD,DATDEB,DATFIN,DEPR,DEPT,DOMEMPL,DOMEMPL_EM,DUREE,FILT,REGR,REGT,SEXE,PCS,TYP_EMPLOI,CONV_COLL,NBHEUR,NBHEUR_TOT,AGE,TRNNETO,TRALCHT,TREFF,CONT_TRAV,CS,AGE_TR,DATDEB_TR,DATFIN_TR,DUREE_TR,POND
0,BE,C5,CG,C,1.0,360.0,49,49,9.0,9.0,360.0,1,52.0,52.0,1,674A,O,292.0,1821.0,1821,44.0,15,99,6.0,CDI,67.0,47.0,1.0,12.0,12.0,12
1,BE,C5,CG,C,31.0,360.0,53,35,9.0,9.0,330.0,1,52.0,53.0,1,674A,O,292.0,1572.0,1731,44.0,12,99,3.0,CDI,67.0,47.0,2.0,12.0,11.0,12
2,BE,C5,CG,C,1.0,360.0,59,59,9.0,9.0,360.0,1,32.0,32.0,1,385B,O,292.0,1825.0,1825,44.0,20,99,3.0,CDI,38.0,47.0,1.0,12.0,12.0,12
3,BE,C5,CG,C,288.0,360.0,62,62,9.0,9.0,73.0,1,32.0,32.0,1,674A,O,292.0,364.0,364,44.0,6,99,4.0,CDD,67.0,47.0,10.0,12.0,3.0,12
4,BE,C5,CG,C,1.0,360.0,66,66,9.0,9.0,360.0,1,76.0,76.0,1,625C,O,292.0,1764.0,1764,44.0,15,99,3.0,CDI,62.0,47.0,1.0,12.0,12.0,12


In [14]:
base.columns

Index(['A6', 'A17', 'A38', 'CPFD', 'DATDEB', 'DATFIN', 'DEPR', 'DEPT',
       'DOMEMPL', 'DOMEMPL_EM', 'DUREE', 'FILT', 'REGR', 'REGT', 'SEXE', 'PCS',
       'TYP_EMPLOI', 'CONV_COLL', 'NBHEUR', 'NBHEUR_TOT', 'AGE', 'TRNNETO',
       'TRALCHT', 'TREFF', 'CONT_TRAV', 'CS', 'AGE_TR', 'DATDEB_TR',
       'DATFIN_TR', 'DUREE_TR', 'POND'],
      dtype='object')

In [19]:
base2.head(100)

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD,TYPE_VAR,LONG_VAR
0,A6,Nomenclature agrégée A6,NaN,Non renseigné,CHAR,2
1,A6,Nomenclature agrégée A6,AZ,"Agriculture, sylviculture et pêche",CHAR,2
2,A6,Nomenclature agrégée A6,BE,"Industrie manufacturière, industries extractiv...",CHAR,2
3,A6,Nomenclature agrégée A6,FZ,Construction,CHAR,2
4,A6,Nomenclature agrégée A6,GI,"Commerce de gros et de détail, transports, héb...",CHAR,2
...,...,...,...,...,...,...
95,DEPR,Département de résidence,26,Drôme,CHAR,3
96,DEPR,Département de résidence,27,Eure,CHAR,3
97,DEPR,Département de résidence,28,Eure-et-Loir,CHAR,3
98,DEPR,Département de résidence,29,Finistère,CHAR,3
